In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
words = open("names.txt", "r").read().split()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
chars=sorted(list(set(''.join(words))))
stoi = {c:i+1 for i , c in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
print(stoi)
print(itos)



{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
block_size = 3 # context length: how many characters to take to predict the next character
X,Y=[],[]
for w in words[:5]:
    print(w)
    context=[0]*block_size # Creates an initial context of [0, 0, 0]. This serves as the starting context before actual characters from the word are added. Assuming 0 is a special padding token
    for ch in w + '.':# Iterate through each character in the word plus a stopping character ('.')
        ix =stoi[ch] # Convert character to its integer index using a predefined `stoi` dictionary
        X.append(context)
        Y.append(ix)
        print(X,Y)
        print(''.join(itos[i] for i in context),'---->',itos[ix])# Print the mapping from context to predicted character
        context=context[1:]+[ix] # Shift the context window by removing the first character and appending the new one
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
[[0, 0, 0]] [5]
... ----> e
[[0, 0, 0], [0, 0, 5]] [5, 13]
..e ----> m
[[0, 0, 0], [0, 0, 5], [0, 5, 13]] [5, 13, 13]
.em ----> m
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13]] [5, 13, 13, 1]
emm ----> a
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1]] [5, 13, 13, 1, 0]
mma ----> .
olivia
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0]] [5, 13, 13, 1, 0, 15]
... ----> o
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15]] [5, 13, 13, 1, 0, 15, 12]
..o ----> l
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15], [0, 15, 12]] [5, 13, 13, 1, 0, 15, 12, 9]
.ol ----> i
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15], [0, 15, 12], [15, 12, 9]] [5, 13, 13, 1, 0, 15, 12, 9, 22]
oli ----> v
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15], [0, 15, 12], [15, 12, 9], [12, 9, 22]] [5, 13, 13, 1, 0, 15, 12, 9, 22, 9]
liv ----> i


In [6]:
X.shape,Y.shape


(torch.Size([32, 3]), torch.Size([32]))

In [7]:
X,Y

(tensor([[ 0,  0,  0],
         [ 0,  0,  5],
         [ 0,  5, 13],
         [ 5, 13, 13],
         [13, 13,  1],
         [ 0,  0,  0],
         [ 0,  0, 15],
         [ 0, 15, 12],
         [15, 12,  9],
         [12,  9, 22],
         [ 9, 22,  9],
         [22,  9,  1],
         [ 0,  0,  0],
         [ 0,  0,  1],
         [ 0,  1, 22],
         [ 1, 22,  1],
         [ 0,  0,  0],
         [ 0,  0,  9],
         [ 0,  9, 19],
         [ 9, 19,  1],
         [19,  1,  2],
         [ 1,  2,  5],
         [ 2,  5, 12],
         [ 5, 12, 12],
         [12, 12,  1],
         [ 0,  0,  0],
         [ 0,  0, 19],
         [ 0, 19, 15],
         [19, 15, 16],
         [15, 16,  8],
         [16,  8,  9],
         [ 8,  9,  1]]),
 tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
          1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0]))

In [26]:
C= torch.randn((27,2))

In [24]:
#F.one_hot(torch.tensor(5),num_classes=27).float() @ C

In [ ]:
e

In [36]:
emb=C[X]
emb.shape

torch.Size([32, 3, 2])

In [ ]:
W1=torch.randn(6,100)
b1=torch.randn(100)
emb @ W1 + b1


In [39]:
torch.flatten(emb,start_dim=1,end_dim=2).shape

torch.Size([32, 6])

In [ ]:
# Function of torch.cat
'''x = torch.randn(2, 3)
print(x)
print(torch.cat((x,x,x),0))
print(torch.cat((x,x),1))
#print(torch.cat((x,x,x),2))'''

tensor([[ 0.0079, -0.0464, -1.6455],
        [-0.4411,  1.0999,  0.4642]])
tensor([[ 0.0079, -0.0464, -1.6455],
        [-0.4411,  1.0999,  0.4642],
        [ 0.0079, -0.0464, -1.6455],
        [-0.4411,  1.0999,  0.4642],
        [ 0.0079, -0.0464, -1.6455],
        [-0.4411,  1.0999,  0.4642]])
tensor([[ 0.0079, -0.0464, -1.6455,  0.0079, -0.0464, -1.6455],
        [-0.4411,  1.0999,  0.4642, -0.4411,  1.0999,  0.4642]])


In [28]:
C[X]

tensor([[[-1.3949,  0.5145],
         [-1.3949,  0.5145],
         [-1.3949,  0.5145]],

        [[-1.3949,  0.5145],
         [-1.3949,  0.5145],
         [ 0.0546,  2.4548]],

        [[-1.3949,  0.5145],
         [ 0.0546,  2.4548],
         [-0.1514, -1.2279]],

        [[ 0.0546,  2.4548],
         [-0.1514, -1.2279],
         [-0.1514, -1.2279]],

        [[-0.1514, -1.2279],
         [-0.1514, -1.2279],
         [ 0.8285,  1.4391]],

        [[-1.3949,  0.5145],
         [-1.3949,  0.5145],
         [-1.3949,  0.5145]],

        [[-1.3949,  0.5145],
         [-1.3949,  0.5145],
         [-0.5391,  0.4790]],

        [[-1.3949,  0.5145],
         [-0.5391,  0.4790],
         [ 0.1167, -0.8259]],

        [[-0.5391,  0.4790],
         [ 0.1167, -0.8259],
         [-0.7695,  0.2704]],

        [[ 0.1167, -0.8259],
         [-0.7695,  0.2704],
         [-0.1459, -1.3000]],

        [[-0.7695,  0.2704],
         [-0.1459, -1.3000],
         [-0.7695,  0.2704]],

        [[-0.1459, -1

#### Now we need to find a way to take the embedding first character of each trigram and form an array that is do this:
 [-1.3949,  0.5145, -1.3949,  0.5145, -1.3949,  0.5145],

 [ 0.0546,  2.4548, -0.1514, -1.2279, -1.3949,  0.5145]
  
 etc 
 
 because the nueral layer is of size (6,100) and the embedding is of size(32,3,2) some how we have to make it (32,6) feed it to the neural layer so to do that there are three ways:

In [61]:
torch.flatten(emb,start_dim=1,end_dim=2).shape

torch.Size([32, 6])

In [ ]:
#Take out the first dimension of the each embedding of a trigram by using unbinding
torch.cat(torch.unbind(emb,1),1)#inefficient

In [ ]:
torch.cat([emb[:,0,:],emb[:,1,:],emb[:,2,:]],dim=1)# unefficient aas creates new tensor to concatenate the values because there is no way to concatenate with using views

tensor([[-1.3949,  0.5145, -1.3949,  0.5145, -1.3949,  0.5145],
        [-1.3949,  0.5145, -1.3949,  0.5145,  0.0546,  2.4548],
        [-1.3949,  0.5145,  0.0546,  2.4548, -0.1514, -1.2279],
        [ 0.0546,  2.4548, -0.1514, -1.2279, -0.1514, -1.2279],
        [-0.1514, -1.2279, -0.1514, -1.2279,  0.8285,  1.4391],
        [-1.3949,  0.5145, -1.3949,  0.5145, -1.3949,  0.5145],
        [-1.3949,  0.5145, -1.3949,  0.5145, -0.5391,  0.4790],
        [-1.3949,  0.5145, -0.5391,  0.4790,  0.1167, -0.8259],
        [-0.5391,  0.4790,  0.1167, -0.8259, -0.7695,  0.2704],
        [ 0.1167, -0.8259, -0.7695,  0.2704, -0.1459, -1.3000],
        [-0.7695,  0.2704, -0.1459, -1.3000, -0.7695,  0.2704],
        [-0.1459, -1.3000, -0.7695,  0.2704,  0.8285,  1.4391],
        [-1.3949,  0.5145, -1.3949,  0.5145, -1.3949,  0.5145],
        [-1.3949,  0.5145, -1.3949,  0.5145,  0.8285,  1.4391],
        [-1.3949,  0.5145,  0.8285,  1.4391, -0.1459, -1.3000],
        [ 0.8285,  1.4391, -0.1459, -1.3

In [ ]:
##Working of torch.unbind
a=torch.randn(2,3,2)
print(a)
torch.unbind(a,dim=2)

tensor([[[-0.0377, -0.0330],
         [-1.5238, -1.3840],
         [ 0.9075,  1.4327]],

        [[ 0.0604, -0.8923],
         [-0.3241,  1.6402],
         [-0.3417,  1.2738]]])


(tensor([[-0.0377, -1.5238,  0.9075],
         [ 0.0604, -0.3241, -0.3417]]),
 tensor([[-0.0330, -1.3840,  1.4327],
         [-0.8923,  1.6402,  1.2738]]))

In [74]:
a=torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [77]:
a.view(2,9)
a.view(2,3,3)
a.view(9,2)


tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17]])

In [ ]:
### Why does this work ?
a.storage()# Number always as a one dimension vector

C:\Users\rohan\AppData\Local\Temp\ipykernel_13376\967240405.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [79]:
emb.view(32,6)

tensor([[-1.3949,  0.5145, -1.3949,  0.5145, -1.3949,  0.5145],
        [-1.3949,  0.5145, -1.3949,  0.5145,  0.0546,  2.4548],
        [-1.3949,  0.5145,  0.0546,  2.4548, -0.1514, -1.2279],
        [ 0.0546,  2.4548, -0.1514, -1.2279, -0.1514, -1.2279],
        [-0.1514, -1.2279, -0.1514, -1.2279,  0.8285,  1.4391],
        [-1.3949,  0.5145, -1.3949,  0.5145, -1.3949,  0.5145],
        [-1.3949,  0.5145, -1.3949,  0.5145, -0.5391,  0.4790],
        [-1.3949,  0.5145, -0.5391,  0.4790,  0.1167, -0.8259],
        [-0.5391,  0.4790,  0.1167, -0.8259, -0.7695,  0.2704],
        [ 0.1167, -0.8259, -0.7695,  0.2704, -0.1459, -1.3000],
        [-0.7695,  0.2704, -0.1459, -1.3000, -0.7695,  0.2704],
        [-0.1459, -1.3000, -0.7695,  0.2704,  0.8285,  1.4391],
        [-1.3949,  0.5145, -1.3949,  0.5145, -1.3949,  0.5145],
        [-1.3949,  0.5145, -1.3949,  0.5145,  0.8285,  1.4391],
        [-1.3949,  0.5145,  0.8285,  1.4391, -0.1459, -1.3000],
        [ 0.8285,  1.4391, -0.1459, -1.3

In [ ]:
W1=torch.randn(6,100)
b1=torch.randn(100)
h=torch.tanh(emb.view(-1,6) @ W1+b1)


In [ ]:
h# nos between -1 and 1 because of tanh


tensor([[-1.0000, -0.9995, -0.0350,  ..., -0.9982, -0.9996, -0.9998],
        [-1.0000, -0.9614, -0.4546,  ..., -0.9965, -1.0000, -1.0000],
        [ 0.9987, -0.9997, -0.4110,  ..., -0.9751, -0.9949,  0.5634],
        ...,
        [ 0.9969, -0.9724, -0.1110,  ..., -0.3865, -0.9794,  0.3704],
        [ 0.0831, -0.9881,  0.3437,  ...,  0.2040, -0.9669, -0.5715],
        [-0.9530,  0.9241,  0.3901,  ..., -0.9862, -0.9713, -0.9972]])

In [86]:
W2=torch.randn(100,27)
b2=torch.randn(27)
logits= h @ W2+b2


In [87]:
counts=logits.exp()
probs=counts/counts.sum(1,keepdim=True)

In [91]:
nll=-probs[torch.arange(32),Y].log().mean()
nll

tensor(15.3648)